# Análise exploratória de dados usando a biblioteca PySpark

## Leonardo Morais Nascimento

## Cientista de Dados

In [ ]:
# Iniciaremos com a instalação da biblioteca PySpark:

%pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=d0d92c3eb94efd3b63d3584b5202592c80874a0c11e1a12ca638e9760ea87b87
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [ ]:
# Instalação das ferramentas necessárias para a análise:

import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import types as T
from pyspark.sql.functions import udf
from pyspark.sql.functions import count_distinct

In [ ]:
# Iniciar uma sessão do Spark:

spark = SparkSession.builder.master('local[*]').getOrCreate()

In [ ]:
# Montagem do Drive:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Utilizaremos o Dataset de vendas "Chipotle" disponível na plataforma Kaggle.

In [ ]:
# Criação do Dataframe:

data_chipotle = spark.read.csv('/content/drive/MyDrive/Data Science/Meu Portifólio/Big Data/Arquivos/chipotle.tsv', sep='\t', header=True)

In [ ]:
# Conhecendo o Dataframe:

data_chipotle.show(15)

+--------+--------+--------------------+--------------------+----------+
|order_id|quantity|           item_name|  choice_description|item_price|
+--------+--------+--------------------+--------------------+----------+
|       1|       1|Chips and Fresh T...|                NULL|    $2.39 |
|       1|       1|                Izze|        [Clementine]|    $3.39 |
|       1|       1|    Nantucket Nectar|             [Apple]|    $3.39 |
|       1|       1|Chips and Tomatil...|                NULL|    $2.39 |
|       2|       2|        Chicken Bowl|[Tomatillo-Red Ch...|   $16.98 |
|       3|       1|        Chicken Bowl|[Fresh Tomato Sal...|   $10.98 |
|       3|       1|       Side of Chips|                NULL|    $1.69 |
|       4|       1|       Steak Burrito|[Tomatillo Red Ch...|   $11.75 |
|       4|       1|    Steak Soft Tacos|[Tomatillo Green ...|    $9.25 |
|       5|       1|       Steak Burrito|[Fresh Tomato Sal...|    $9.25 |
|       5|       1| Chips and Guacamole|           

Quantos objetos existem no Dataframe?

In [ ]:
# Contagem de objetos do Dataframe:

data_chipotle.count()

4622

In [ ]:
# Quais as colunas do Dataframe?

# Impressão das colunas do Dataframe

col_name = data_chipotle.columns
print(f"Colunas do DataFrame : {col_name}")

Colunas do DataFrame : ['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']


In [ ]:
# Queremos conhecer as informações das colunas do Dataframe:

data_chipotle.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: string (nullable = true)



Para o seguimento da análise devemos converter o tipo da coluna "item_price" de 'string' para 'float'.

In [ ]:
# Conversão da coluna 'item_price' para o tipo float:

data_chipotle02 = data_chipotle\
.withColumn('item_price', F.regexp_replace('item_price', '[$,]', '').cast('float'))\
.withColumn('quantity', data_chipotle['quantity'].cast('float'))
data_chipotle02.printSchema()

root
 |-- order_id: string (nullable = true)
 |-- quantity: float (nullable = true)
 |-- item_name: string (nullable = true)
 |-- choice_description: string (nullable = true)
 |-- item_price: float (nullable = true)



Qual item foi o mais vendido?

In [ ]:
#  Contaremos a quantidade que cada item foi vendido no Dataframe:

data_pedidos = data_chipotle.groupBy("item_name").count()
data_pedidos.show()

+--------------------+-----+
|           item_name|count|
+--------------------+-----+
| Carnitas Soft Tacos|   40|
|  Chicken Soft Tacos|  115|
|               Salad|    2|
|         Steak Salad|    4|
|                Bowl|    2|
| Veggie Crispy Tacos|    1|
|   6 Pack Soft Drink|   54|
|Chicken Crispy Tacos|   47|
|   Veggie Soft Tacos|    7|
| Carnitas Salad Bowl|    6|
|     Chicken Burrito|  553|
|   Veggie Salad Bowl|   18|
|         Veggie Bowl|   85|
|       Steak Burrito|  368|
|  Steak Crispy Tacos|   35|
|   Canned Soft Drink|  301|
|        Chicken Bowl|  726|
|       Side of Chips|  101|
|                Izze|   20|
|  Chicken Salad Bowl|  110|
+--------------------+-----+
only showing top 20 rows



In [ ]:
# Mudaremos para decrescente a ordem dos itens:
data_pedidos.orderBy('count' , ascending=False).show()

+--------------------+-----+
|           item_name|count|
+--------------------+-----+
|        Chicken Bowl|  726|
|     Chicken Burrito|  553|
| Chips and Guacamole|  479|
|       Steak Burrito|  368|
|   Canned Soft Drink|  301|
|               Chips|  211|
|          Steak Bowl|  211|
|       Bottled Water|  162|
|  Chicken Soft Tacos|  115|
|  Chicken Salad Bowl|  110|
|Chips and Fresh T...|  110|
|         Canned Soda|  104|
|       Side of Chips|  101|
|      Veggie Burrito|   95|
|    Barbacoa Burrito|   91|
|         Veggie Bowl|   85|
|       Carnitas Bowl|   68|
|       Barbacoa Bowl|   66|
|    Carnitas Burrito|   59|
|    Steak Soft Tacos|   55|
+--------------------+-----+
only showing top 20 rows



O item mais vendido foi Chicken Bowl com 726 itens.

Qual o total de entrada no caixa?

In [ ]:
# Devemos criar uma nova coluna com o produto das colunas 'Quantity' e 'Item_price':

data_chipotle03 = data_chipotle02.withColumn('Total_venda',data_chipotle02['quantity'] * data_chipotle02['item_price'])
data_chipotle03.show()

+--------+--------+--------------------+--------------------+----------+-----------+
|order_id|quantity|           item_name|  choice_description|item_price|Total_venda|
+--------+--------+--------------------+--------------------+----------+-----------+
|       1|     1.0|Chips and Fresh T...|                NULL|      2.39|       2.39|
|       1|     1.0|                Izze|        [Clementine]|      3.39|       3.39|
|       1|     1.0|    Nantucket Nectar|             [Apple]|      3.39|       3.39|
|       1|     1.0|Chips and Tomatil...|                NULL|      2.39|       2.39|
|       2|     2.0|        Chicken Bowl|[Tomatillo-Red Ch...|     16.98|      33.96|
|       3|     1.0|        Chicken Bowl|[Fresh Tomato Sal...|     10.98|      10.98|
|       3|     1.0|       Side of Chips|                NULL|      1.69|       1.69|
|       4|     1.0|       Steak Burrito|[Tomatillo Red Ch...|     11.75|      11.75|
|       4|     1.0|    Steak Soft Tacos|[Tomatillo Green ...|    

In [ ]:
# Agregar os itens da coluna 'Total_venda' pela soma dos valores:

data_chipotle03.agg({'Total_venda': 'sum'}).show()

+----------------+
|sum(Total_venda)|
+----------------+
|39237.0197327137|
+----------------+



O valor total das vendas é de 39.237

Qual o valor de entrada médio?

In [ ]:
data_chipotle03.agg({'Total_venda': 'mean'}).show()

+-----------------+
| avg(Total_venda)|
+-----------------+
|8.489186441521786|
+-----------------+



Qual a quantidade de itens diferentes vendidos?

In [ ]:
data_chipotle03.select(count_distinct("item_name")).show()

+-------------------------+
|count(DISTINCT item_name)|
+-------------------------+
|                       50|
+-------------------------+



Quais produtos tem valor superior a 10,00?

In [ ]:
# Classificaremos os produtos criando uma coluna de acordo com o valor se for acima ou abaixo de 10:

maior_10 = data_chipotle03.withColumn("classificacao",
                 F.when(F.col("item_price") > F.lit(10),
                        F.lit("acima de 10"))\
                 .otherwise(F.lit("abaixo de 10")))

maior_10.show()

+--------+--------+--------------------+--------------------+----------+-----------+-------------+
|order_id|quantity|           item_name|  choice_description|item_price|Total_venda|classificacao|
+--------+--------+--------------------+--------------------+----------+-----------+-------------+
|       1|     1.0|Chips and Fresh T...|                NULL|      2.39|       2.39| abaixo de 10|
|       1|     1.0|                Izze|        [Clementine]|      3.39|       3.39| abaixo de 10|
|       1|     1.0|    Nantucket Nectar|             [Apple]|      3.39|       3.39| abaixo de 10|
|       1|     1.0|Chips and Tomatil...|                NULL|      2.39|       2.39| abaixo de 10|
|       2|     2.0|        Chicken Bowl|[Tomatillo-Red Ch...|     16.98|      33.96|  acima de 10|
|       3|     1.0|        Chicken Bowl|[Fresh Tomato Sal...|     10.98|      10.98|  acima de 10|
|       3|     1.0|       Side of Chips|                NULL|      1.69|       1.69| abaixo de 10|
|       4|

In [ ]:
# Contagem do atributos da coluna 'Classificação':

maior_10.groupBy("classificacao").count().show()

+-------------+-----+
|classificacao|count|
+-------------+-----+
|  acima de 10| 1130|
| abaixo de 10| 3492|
+-------------+-----+



Qual é o preço de cada item?

In [ ]:
# Criaremos um dataframe com o nome e preço dos itens individualizado:

item_cada = data_chipotle03.dropDuplicates(["item_name"])
item_cada = item_cada.drop("order_id", "quantity", "choice_description", "Total_venda", "classificacao" )
item_cada.orderBy('item_name').show()

+--------------------+----------+
|           item_name|item_price|
+--------------------+----------+
|   6 Pack Soft Drink|      6.49|
|       Barbacoa Bowl|     11.75|
|    Barbacoa Burrito|      8.99|
|Barbacoa Crispy T...|     11.75|
| Barbacoa Salad Bowl|     11.89|
| Barbacoa Soft Tacos|      9.25|
|       Bottled Water|      1.09|
|                Bowl|      22.2|
|             Burrito|       7.4|
|         Canned Soda|      2.18|
|   Canned Soft Drink|      1.25|
|       Carnitas Bowl|      8.99|
|    Carnitas Burrito|      8.99|
|Carnitas Crispy T...|      9.25|
|      Carnitas Salad|      8.99|
| Carnitas Salad Bowl|     11.89|
| Carnitas Soft Tacos|      9.25|
|        Chicken Bowl|     16.98|
|     Chicken Burrito|      8.49|
|Chicken Crispy Tacos|      8.75|
+--------------------+----------+
only showing top 20 rows



Qual a quantidade de vendas por item?

In [ ]:
# Quantidade de vendas do item 'Veggie Salad Bowl'

data_filtro = data_chipotle03.filter(F.col("item_name") == "Veggie Salad Bowl").count()
data_filtro

18

Quantas vezes alguém pediu um item mais de uma vez?

In [ ]:
# Vezes que alguém pediu o item 'Canned Soda' mais de uma vez

data_soda = data_chipotle03.filter(F.col("item_name") == "Canned Soda").filter(F.col("quantity") > 1).count()
data_soda

20

Análise exploratória usando o PySpark

Leonardo Morais Nascimento